# Generalisation

In [4]:
import pandas as pd                                        
import numpy as np
from math import isnan
import sys
from io import StringIO
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Markdown, clear_output 
import ipywidgets as widgets
from sklearn.preprocessing import LabelEncoder
import random
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
import progressbar 
import time
import seaborn as sns
%matplotlib inline

############################################################
name_file='Dataset_9_cost.csv'
def df_create(name=""):
    try:
        df=pd.read_csv(name,sep='|')
        #@variable_global : 
        global nb_Columns 
        global nb_Lines 
        global list_Unnamed 
        global list_Named 
        
        nb_Columns=df.shape[1] #nombre de colonnes du df "nb_Columns=16"
        nb_Lines=df.shape[0] #nombre de lignes du df "nb_lines=1338"
        list_Unnamed=df.columns[0:9].tolist()#liste des colonnes d'index 0 à 9 "les colonnes commençant par Unnamed"
        list_Named=df.columns[9:nb_Columns].tolist()#liste des colonnes d'index 9 à 16 "les colonnes ne commançant pas par Unnamed"
    except:
        print('Fichier inconnu') #erreur retourner si la lecture du df est erroné
    return df
############################################################
def analyse_type_variable(df):#Analyse et calcule de pourcentage type de varibaible"Null,String,Integer,Float,Other"
    dic={}
    try:
        for col_name in df.columns:
            nb_integer,nb_float,nb_string,nb_boolean,nb_null,nb_other=0,0,0,0,0,0
            
            for val in df[col_name]:
                try:     
                    if type(float(val))==float:
                        try:
                            if len(str(int(val)))==len(str(val)):
                                nb_integer +=1
                            else:
                                nb_float +=1
                        except:
                            if isnan(val):
                                nb_null+=1
                            else:
                                nb_float +=1
                    elif isnan(val):
                        nb_null+=1
                except:
                    try:
                        if type(val)==str:
                            nb_string+=1
                    except:
                        nb_other+=1
            dic.update({col_name:{'Integer':{'nb':nb_integer,'percentage':(nb_integer/nb_Lines)*100}}})
            dic[col_name].update({'Float':{'nb':nb_float,'percentage':(nb_float/nb_Lines)*100}})
            dic[col_name].update({'String':{'nb':nb_string,'percentage':(nb_string/nb_Lines)*100}})
            dic[col_name].update({'Boolean':{'nb':nb_boolean,'percentage':(nb_boolean/nb_Lines)*100}})
            dic[col_name].update({'NaN':{'nb':nb_null,'percentage':(nb_null/nb_Lines)*100}})
            dic[col_name].update({'Other':{'nb':nb_other,'percentage':(nb_other/nb_Lines)*100}})

    except:
        print('[Data] incorrect (function analyse_type_variable)')
    
    return dic

############################################################################################
def splot_donuts(dic):
    ig, ax = plt.subplots(figsize=(19, 4), subplot_kw=dict(aspect="equal"))

    list_types,list_percentage,nb_types,list_nb=[],[],[],[]
    
    for key,val in dic.items():
        list_percentage.append(val['percentage'])
        list_types.append(key+" ==> "+str(round(val['percentage'],2))+"%")
        nb_types.append(key+" ==> "+str(val['nb']))
        list_nb.append(int(val['nb']))
        
    wedges, texts = ax.pie(list_percentage, wedgeprops=dict(width=0.5), startangle=-40)

    bbox_props = dict(boxstyle="square,pad=0.3", fc="w", ec="k", lw=0.72)
    kw = dict(arrowprops=dict(arrowstyle="-"),
              bbox=bbox_props, 
              zorder=0, va="center")
    
    ax.legend(wedges, list_types,
          title="Types",
          loc="center left",
          bbox_to_anchor=(1, 0, 0.5, 1))
    
    for i, p in enumerate(wedges):
        ang = (p.theta2 - p.theta1)/2. + p.theta1
        y = np.sin(np.deg2rad(ang))
        x = np.cos(np.deg2rad(ang))
        
        horizontalalignment = {-1: "right", 1: "left"}[int(np.sign(x))]
        connectionstyle = "angle,angleA=0,angleB={}".format(ang)
        kw["arrowprops"].update({"connectionstyle": connectionstyle})
        
        if i < len(nb_types):
            if list_nb[i]>0:
                ax.annotate(nb_types[i], xy=(x, y), xytext=(1.35*np.sign(x), 1.4*y),
                    horizontalalignment=horizontalalignment, **kw)
    plt.show()
#############################################################################################
def description_col(col_name,dic_col):
    desc_col=[]
    if col_name in list_Unnamed:
        desc_col.append("La colonne ["+str(col_name)+"] est une colonne inconnu")
        desc_col.append("Elle est composé de:")
        for key,val in dic_col[col_name].items():
            if val['nb']!=0:
                desc_col.append("-"+str(val['nb'])+" "+str(key)+" soit "+str(round(val['percentage'],2))+"% des types de variable")
    elif col_name in list_Named:
        desc_col.append("La colonne ["+str(col_name)+"] est une colonne principale")
        desc_col.append("Elle est composé de:")
        for key,val in dic_col[col_name].items():
            if val['nb']!=0:
                desc_col.append("-"+str(val['nb'])+" "+str(key)+" soit "+str(round(val['percentage'],2))+"% des types de variable")
                    
    return desc_col
#############################################################################################
list_sex=['male','female']
list_smoker=['yes','no']
list_region=['southwest','southeast','northwest','northeast']
list_children=['0','1','2','3','4','5']
#############################################################################################
def filter_1(df,lst,colname):#filtrage des string non appartenant au liste ci dessus
    for i in range(nb_Lines):
        try:
            if str(df[colname][i]) not in lst :
                df[colname][i]=np.nan     
        except:
            print('Erreur Colonne:'+colname+' Indice:'+i+' Valeur:'+str(df[colname][i])+' (function filter_1)')
    return df
########################################################################################################
def filter_2(df="",col_name=""):#filtrage des float et int 
    if(col_name!=""):
        try:
            for i in range(nb_Lines):
                
                try:
                    float(df[col_name][i])
                    if col_name=="age":
                        if float(df[col_name][i])<=0 or float(df[col_name][i]) > 140:
                            df[col_name][i]=np.nan
                    else:
                        if float(df[col_name][i])<=0 :
                            df[col_name][i]=np.nan
                except:
                    df[col_name][i]=np.nan
        except:
            print("[Data] ou [Colonne] incorrecte (function filter_2)")
    else: 
        print("Vous devez entrer [Data] et [Colonne] (function filter_2)")
    return df

############################################################
def nan_to_value(df="",col_name="",type_col=""):
    try:
        med=df[col_name][df[col_name].notnull()].astype(type_col).mean()
        for i in range(nb_Lines):
            try:
                if isnan(df[col_name][i]):
                    df[col_name][i]=round(med,1)
            except:
                df[col_name][i]=round(float(df[col_name][i]),3)
    except:
            print("[Data] ou [Colonne] incorrecte")
    df[col_name] = df[col_name].astype(type_col)
############################################################
def scatterplot_description(df,col_name1,col_name2):
    sns.set_style("whitegrid", {'grid.linestyle': '--'})
    plt.figure(figsize = (10,4))
    colonne=df[col_name1].unique()
    plt.title("Description des charges"+" par "+ col_name1 +" et "+ col_name2)
    for i, colonne in enumerate(colonne):
        temp = df[df[col_name1] == colonne]
        sns.scatterplot(temp[col_name2],temp["charges"])
    plt.legend(df[col_name1].unique())
############################################################
def boxplot_description(df,col_name1,col_name2):
    plt.figure(figsize = (10,6))
    plt.title("Description charges"+" par "+ col_name1 +" et "+ col_name2)
    sns.boxplot(x = col_name2, y = "charges", hue = col_name1, data = df)
############################################################
def random_tree(df):
    X = np.array(df.drop(['charges'],1))
    y = df['charges']
    X_trainC , X_testC,y_trainC , y_testC =train_test_split(X,y,test_size=0.2)
    clf = tree.DecisionTreeRegressor()
    clf.fit(X_trainC,y_trainC)
    charges_accuracy_tree=clf.score(X_testC,y_testC)
    print(clf.predict(X_testC))
    print('charges accuracy score using Tree regression: ',charges_accuracy_tree*100)
############################################################
    
def decision_tree(df,col_name):
    X = np.array(df.drop(['charges'],1))
    try:
        clf = DecisionTreeClassifier().fit(np.array(df.drop(['charges'],1)), df[col_name])
        plt.figure(figsize = (15,6))
        plt.suptitle("Decision surface of a decision tree using paired features")
        plot_tree(clf, filled=True)
        plt.show()
    except:
        print('L\'arbre de décision ne peut pas se faire sur la colonne ['+col_name+']')
############################################################
def animated_marker(): 
    widgets = ['Loading: ', progressbar.AnimatedMarker()] 
    bar = progressbar.ProgressBar(widgets=widgets).start() 
    for i in range(10): 
        time.sleep(0.1) 
        bar.update(i)

############################################################
def random_color():
     return "#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
############################################################    
def color(val):
    try:
        if isnan(val):
            return 'background-color: %s' %'red'
    except:   
        return 'background-color: %s'% random_color()
############################################################   
def style_df(df):
    df=df.astype(str)
    return df.style.applymap(color, subset=list(range(df.shape[1])))
############################################################
def application(df):
    col=""
    tab = widgets.Tab()  
    tab.df=df
    tab.encoder=0
    tab.traitment=0
    tab.decision=0
    out  = widgets.Output(layout={'border': '1px solid black'})
    out2 = widgets.Output(layout={'border': '1px solid black'})
    out3 = widgets.Output(layout={'border': '1px solid black'})
    out4 = widgets.Output(layout={'border': '1px solid black'})
    out5 = widgets.Output(layout={'border': '1px solid black'})
    out6 = widgets.Output(layout={'border': '1px solid black'})
    up = widgets.FileUpload(accept="", multiple=False)

    delim = widgets.RadioButtons(
        options=[';', ',', '|',' '],
        description='Separator: ',
        disabled=False)

    eraser = widgets.SelectMultiple(
        options=['tab','"'],
        value=['tab'],
        #rows=10,
        description='Eraser: ',
        disabled=False)

    rows = widgets.IntSlider(
        value=0,
        step=1,
        description='# of lines:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='d')

    button_upload = widgets.Button(
        description='Upload',
        disabled=False,
        button_style='warning',
        tooltip='Click to Upload',
        icon='check')

    button_preview = widgets.Button(
        description='Preview',
        disabled=False,
        button_style='info',
        tooltip='Click to Preview',
        icon='search')

    button_plot = widgets.Button(
        description='Plot',
        disabled=False,
        button_style='danger',
        tooltip='Click to Plot',
        icon='pencil')
    
    button_random_tree = widgets.Button(
        description='Random Tree',
        disabled=False,
        button_style='warning',
        tooltip='Click to show random tree',
        icon='pencil')
    
    graph_type = widgets.Dropdown(
        options=['Scatter Plot','Box Plot','Bar Chart', 'Line Chart'],
        value='Scatter Plot',
        description='Chart Type:',
        disabled=False)
    
    x_axis = widgets.Dropdown(
        options=[''],
        value='',
        description='X-Axis:',
        disabled=False)

    y_axis = widgets.Dropdown(
        options=[''],
        value='',
        description='Y-Axis:',
        disabled=False)

    color_picker = widgets.ColorPicker(
        concise=False,
        description='Color Picker: ',
        value='lightblue',
        disabled=False)
    
    button_encoder= widgets.Button(
        description='Encoder',
        disabled=False,
        button_style='info',
        tooltip='Click to encode columns',
        icon='pencil')
    
    button_decision = widgets.Button(
        description='Decision Tree',
        disabled=False,
        button_style='success',
        tooltip='Click to show the decision tree'
 )
    
    accordion_plot = widgets.Accordion(children=[
          widgets.VBox([
            widgets.HBox([graph_type, color_picker]),
            widgets.HBox([x_axis, y_axis]), 
            widgets.HBox([button_plot,button_decision,button_random_tree])
              ,out3]),
          widgets.VBox([widgets.HBox([button_encoder]),out4])
    ])
    
    accordion_plot.set_title(0, 'Plot')
    accordion_plot.set_title(1, 'Encoder')
    accordion_box_plot=widgets.VBox([
        accordion_plot
    ])

    button_detail = widgets.Button(
        description='Details',
        disabled=False,
        button_style='info',
        tooltip='Click to see details')

    button_correction = widgets.Button(
        description='DataFrame',
        disabled=False,
        button_style='success',
        tooltip='Click to correct DataFrame',
        icon='pencil')

    button_delete = widgets.Button(
        description='Duplicate Lines',
        disabled=False,
        button_style='danger',
        tooltip='Click to delete duplicate lines',
        icon='trash')
    
    button_delete_columns = widgets.Button(
        description='Unnamed Columns',
        disabled=False,
        button_style='danger',
        tooltip='Click to delete unnamed columns',
        icon='trash')

    button_corr = widgets.Button(
        description='Correlation',
        disabled=False,
        button_style='info',
        tooltip='Click to show correlation',
        icon='search')

    
    menu = widgets.Dropdown(
               options=tab.df.columns.tolist(),
               value='age',
               description='Columns:')
    
    toggle = widgets.ToggleButtons(
        options=['Preview  ', 'Info  ', 'Stats  ',],
        description='Options',
        disabled=False,
        button_style='warning',
        icons=['search', 'info', 'tachometer'])
    
    accordion_describer = widgets.Accordion(children=[
        widgets.VBox([toggle, out]), 
        widgets.VBox([menu, widgets.HBox(
            [button_detail]),out6]),
            widgets.VBox([widgets.HBox(
            [button_corr,button_delete,button_delete_columns,button_correction]),out2])])
    
    accordion_describer.set_title(0, 'Options')
    accordion_describer.set_title(1, 'Details')
    accordion_describer.set_title(2, 'Edit')
    accordion_box_desc=widgets.VBox([
        accordion_describer
    ])
    accordion = widgets.Accordion(children=[

        widgets.VBox([delim, eraser]), 
        rows])

    accordion.set_title(0, 'File Selection')
    accordion.set_title(1, 'Delimiter')
    accordion.set_title(2, 'Skip Rows')

    accordion_box = widgets.VBox([
        widgets.HBox([button_preview, button_upload]),
        out
    ])


    children = [
        accordion_box, 
        accordion_box_desc,
        accordion_box_plot,
          ]

    tab.children = children
    tab.set_title(0, "Upload")
    tab.set_title(1, "Describer")
    tab.set_title(2, "Plotter")


    def content_parser():
        if up.value == {}:
            with out:
                print('No CSV loaded')    
        else:
            typ, content = "", ""
            up_value = up.value
            for i in up_value.keys():
                typ = up_value[i]["metadata"]["type"]
                if typ == "text/csv":
                    content = up_value[i]["content"]
                    content_str = str(content, 'utf-8')

                if eraser.value != {}: 
                    for val in eraser.value:
                        if val == "tab":
                            content_str = content_str.replace("\t","")
                        else:
                            content_str = content_str.replace(val,"")
                if content_str != "":
                    str_io = StringIO(content_str) 
                    return str_io
    def df_converter():
        #content = content_parser()
        #if content is not None:
            #df = pd.read_csv(content, sep=delim.value, index_col=False, skiprows=rows.value)
        return tab.df.copy()
        #else:
            #return None
    def preview():
        df = df_converter()
        with out:
            out.clear_output()
            print('\n -----Now this is how your DF looks like:----- \n')
            if df is not None:
                display(df.loc[:9].style.background_gradient(cmap='viridis').highlight_null("pink"))
            else:
                print('Configuration is wrong/missing...')
    def upload():
        df = df_converter()
        with out:
            out.clear_output()
            if df is not None:
                print('\n --------Your DF is uploaded :-------- \n')
                x_axis.options = df.columns
                y_axis.options = df.columns
            else:
                print('Configuration is wrong/missing...')
    def desc():
        info_level = toggle.value
        if info_level != {}:
            df = df_converter()
            with out:
                out.clear_output()
                print('\n ------Your {} looks like:------ \n'.format(
                    info_level))
                if df is not None:
                    if info_level == 'Info  ':
                        print(df.info(verbose=True))
                    elif info_level == 'Stats  ':
                        print(df.describe())
                    elif info_level == 'Preview  ':
                        preview()
                    else:
                        print('Configuration is wrong/missing...')
    
    def plot():
            graph = graph_type.value
            button_decision.disabled=False
            x_axis.disabled=False
            graph_type.disabled=False
            color_picker.disabled=False
            button_random_tree.disabled=False
            y_axis.description='Y-Axis:'
            if graph != {}:
                df = df_converter()
                with out3:
                    out3.clear_output()
                    print('\n ------Your {} looks like:------ \n'.format(graph))
                    if (df is not None):
                        
                        try:
                            df = df.head(5)
                            height = df[y_axis.value]
                            bars = df[x_axis.value]
                            y_pos = np.arange(len(height))
                            if tab.decision==1:
                                try:
                                    with out3:
                                        out3.clear_output()
                                        decision_tree(df,y_axis.value)
                                        tab.decision=0
                                except:
                                    print('Axis [X,Y] not compatible') 
                            else:
                                if graph == 'Bar Chart':
                                    with out3:
                                        out3.clear_output()
                                        plt.figure(figsize=(10,4))
                                        plt.bar(
                                            y_pos, 
                                            height, 
                                            color=color_picker.value)
                                        plt.xticks(y_pos, bars)
                                        plt.show()
                                elif graph == 'Line Chart':
                                    with out3:
                                        out3.clear_output()
                                        plt.figure(figsize=(10,4))
                                        plt.plot(
                                            bars,
                                            height, 
                                            color=color_picker.value,
                                            marker='o', 
                                            linestyle='solid'
                                        )
                                        plt.xticks(bars)
                                        plt.show()
                                elif graph == 'Scatter Plot':
                                    if tab.encoder ==0:
                                        print('You must encode DF')
                                    else:
                                        try:
                                            with out3:
                                                out3.clear_output()
                                                scatterplot_description(df_converter(),x_axis.value,y_axis.value)
                                                plt.show()
                                        except:
                                            print('Axis [X,Y] not compatible')
                                elif graph == 'Box Plot':
                                    if tab.encoder ==0:
                                        print('You must encode DF')
                                    else:
                                        try:
                                            with out3:
                                                out3.clear_output()
                                                boxplot_description(df_converter(),x_axis.value,y_axis.value)
                                                plt.show()
                                        except:
                                            print('Axis [X,Y] not compatible')
                                   
                        except:
                            print('You must Upload DF')
                    
    def correction():
        with out2:
                clear_output()
                if  tab.traitment==0 :
                    df = df_converter()
                    animated_marker()
                    filter_1(df, list_sex,"sex")
                    filter_1(df, list_smoker,"smoker")
                    filter_1(df, list_region,"region")
                    filter_1(df, list_children,"children")

                    filter_2(df,"age")
                    nan_to_value(df,'age','int64')
                    filter_2(df,"bmi")
                    nan_to_value(df,'bmi','float64')
                    filter_2(df,"charges")
                    nan_to_value(df,'charges','float64')
                    tab.df=df.copy()
                    upload()
                    tab.traitment=1
                else:
                    print('Correction of Df is alredy applicated')
                
    def detail():
        with out6:
                clear_output()
                df = df_converter()
                dic_col=analyse_type_variable(df)
                menu.options=df.columns.tolist()
                col=menu.value
                splot_donuts(dic_col[col])
                if len(children)==4:
                    children.pop()
                desc_col_list=description_col(col,dic_col)
                description_columns=[]
                for des_val in desc_col_list:
                    description_columns.append(widgets.Label(value=str(des_val)))

                box2 = widgets.VBox(description_columns)

                children.append(box2)
                tab.children = children
                tab.set_title(3, "Description")
    def delete():
        with out2:
                clear_output()
                df = df_converter()
                animated_marker() 
                try:
                    dup_drop=df.drop(list_Unnamed, axis = 1).drop_duplicates()

                    if(dup_drop.shape[0]==nb_Lines):
                        print('Aucune ligne duppliquer')
                    else:
                        print(str(nb_lines-dup_drop.sahpe[0])+" lignes supprimés")
                        menu.options=df.columns.tolist()
                        tab.df=df.copy()
                        upload()
                except:
                    print('Aucune ligne duppliquer')
                    
    def delete_columns():
        with out2:
            clear_output()
            df = df_converter()
            animated_marker() 
            try:
                df = df.drop(columns=list_Unnamed)
                print(str(len(list_Unnamed))+" colonnes supprimés")
                    
            except:
                print("Aucune colonne a supprimé")
            menu.options=df.columns.tolist()
            tab.df=df.copy()
            upload()
            
    def encoder():
         with out4:
            clear_output()
            df = df_converter()
            animated_marker()
            sex_arr=np.array([df['sex'].unique()])
            children_arr=np.array([df['children'].unique()])
            region_arr=np.array([df['region'].unique()])
            smoker_arr=np.array([df['smoker'].unique()])

            sex_df=pd.DataFrame(data=sex_arr,index=['Sex'])
            children_df=pd.DataFrame(data=children_arr,index=['Children'])
            region_df=pd.DataFrame(data=region_arr,index=['Region'])
            smoker_df=pd.DataFrame(data=region_arr,index=['Smoker'])
            animated_marker()
            print('\n  ------------ Before ------------ \n')
            display(style_df(sex_df))
            display(style_df(children_df))
            display(style_df(region_df))
            display(style_df(smoker_df))
            
            sex=LabelEncoder()
            df['sex']=sex.fit_transform(df['sex'].astype(str))
            
            region=LabelEncoder()
            df['region']=region.fit_transform(df['region'].astype(str))

            smoker=LabelEncoder()
            df['smoker']=smoker.fit_transform(df['smoker'].astype(str))

            children=LabelEncoder()
            df['children']=children.fit_transform(df['children'].astype(str))
            
            sex_arr=np.array([df['sex'].unique()])
            children_arr=np.array([df['children'].unique()])
            region_arr=np.array([df['region'].unique()])
            smoker_arr=np.array([df['smoker'].unique()])

            sex_df=pd.DataFrame(data=sex_arr,index=['Sex'])
            children_df=pd.DataFrame(data=children_arr,index=['Children'])
            region_df=pd.DataFrame(data=region_arr,index=['Region'])
            smoker_df=pd.DataFrame(data=region_arr,index=['Smoker'])
            animated_marker()
            print('\n ------------ After ------------ \n')
            display(style_df(sex_df))
            display(style_df(children_df))
            display(style_df(region_df))
            display(style_df(smoker_df))
            tab.df=df.copy()
            tab.encoder=1
            upload()
    def corr():
         with out2:
            out2.clear_output()
            df = df_converter()
            plt.figure(figsize=(10,4))
            sns.heatmap(df.corr(), annot = True)
            plt.show()
    def decision():
         with out3:
            out3.clear_output()
            graph = graph_type.value
            if graph != {}:
                df = df_converter()
                if (df is not None):
                    try:
                        df = df.head(5)
                        height = df[y_axis.value]
                        bars = df[x_axis.value]
                        y_pos = np.arange(len(height))
                        if tab.encoder ==0:
                            print('You must encode DF')
                        else:
                            try:
                                with out3:
                                    out3.clear_output()
                                    x_axis.disabled=True
                                    graph_type.disabled=True
                                    color_picker.disabled=True
                                    y_axis.description='Columns'
                                    button_decision.disabled=True
                                    button_random_tree.disabled=True
                                    tab.decision=1
                            except:
                                print('Axis [X,Y] not compatible') 
                    except:
                        print('You must Upload DF')
    def random_T():
         with out3:
            out3.clear_output()
            df = df_converter()
            if (df is not None):
                try:
                    if tab.encoder ==0:
                        print('You must encode DF')
                    else:
                        try:
                            with out3:
                                out3.clear_output()
                                random_tree(df)
                        except:
                            print('Data Contain Errors') 
                except:
                    print('Data Contain Errors')
        
        
    def preview_clicked(b):
        preview()
    def upload_clicked(b):
        upload()
    def desc_clicked(b):
        desc()
    def plotter_clicked(b):
        plot()
    def correction_clicked(b):
        correction() 

    def detail_clicked(b):
            detail()

    def delete_clicked(b):
        delete()
    def delete_columns_clicked(b):
        delete_columns()
    def encoder_clicked(b):
        encoder()
    def corr_clicked(b):
        corr()
    def decision_clicked(b):
        decision()       
    def random_tree_clicked(b):
        random_T()          
    button_preview.on_click(preview_clicked)
    button_upload.on_click(upload_clicked)
    toggle.observe(desc_clicked, 'value')
    button_plot.on_click(plotter_clicked)
    button_detail.on_click(detail_clicked)
    button_delete.on_click(delete_clicked)
    button_delete_columns.on_click(delete_columns_clicked)
    button_correction.on_click(correction_clicked)
    button_encoder.on_click(encoder_clicked)
    button_corr.on_click(corr_clicked)
    button_decision.on_click(decision_clicked)
    button_random_tree.on_click(random_tree_clicked)
    return tab

#########################EXECUTION###################################
application(df_create(name_file))